In [ ]:
from pathlib import Path


datapath = Path("data")

# Dados tipo vetor


<div class="alert-info">

### Perguntas
- O que são dados vetoriais no contexto SIG?
- Quais as diferenças entre dados vetoriais e raster?
- Quando usar um ou o outro?

### Objetivos
- Leitura e manipulação de dados vetoriais.
- Entender o formato e suas utilidades.

</div>

Dados vetorias são pontos ou conjunto de pontos em um plano geométrico. Dados vetorias geospaciais o plano geométrico e definido por um sistema de coordenadas.


Usaremos principalmente o `geopandas` para manipular os dados vetorais, mas antes de chegar lá vamos revisar brevemente o "stack" em que o geopandas é contruído.

- geos &rarr; pygeos e shapely
- gdal &rarr; fiona
- pandas
- tudo junto &rarr; geopandas

### geos

In [ ]:
from pygeos import Geometry


geometry = Geometry.from_wkt("POINT (5.2 52.1)")
geometry

In [ ]:
from pygeos import points

point = points(5.2, 52.1)

In [ ]:
from shapely.geometry import box

b = box(0.0, 0.0, 1.0, 1.0)
b

In [ ]:
b.centroid

In [ ]:
b.centroid.coords.xy, b.area

### gdal

In [ ]:
import fiona


coll = fiona.open(datapath.joinpath("floripa.geojson"))

coll

In [ ]:
coll.meta

In [ ]:
coll.bounds, coll.crs

In [ ]:
for k, item in coll.items():
    print(item)

### GeoPandas

In [ ]:
import geopandas


gdf = geopandas.read_file(datapath.joinpath("floripa.geojson"))
gdf

<div class="alert-warning">

Exercício:

1. Usando o `repr` do Jupyter mostre um item da coluna `geometry`.

</div>

```python
gdf[""]
```

### Pausa para falar sobre os tipos fundamentais de vetores!

### Pontos

In [ ]:
import geopandas
from shapely.geometry import Point
from toolz.dicttoolz import valmap


pybrs = {
    "Campinas": [-47.060833, -22.905833],
    "Brasília": [-47.882778, -15.793889],
    "Joinville": [-48.845833, -26.303889],
    "Rio de Janeiro": [-43.196389, -22.908333],
    "Caxias do Sul": [-51.178889, -29.167778],
    "Curitiba": [-49.271944, -25.429722],
    "São Paulo": [-46.633333, -23.55],
    "Cidade Nova": [-43.2025, -22.91],
    "Brasília": [-47.882778, -15.793889],
    "Porto de Galinhas": [-35.0011, -8.5041],
    "São José dos Campos": [-45.886944, -23.178889],
    "Florianópolis": [-48.548889, -27.596944],
    "Belo Horizonte": [-43.933333, -19.916667],
    "Ribeirão Preto": [-47.81, -21.177778],
}


pybrs_points = valmap(Point, pybrs)

gs = geopandas.GeoSeries(pybrs_points)
gs.crs = {'init': 'epsg:4326'}
gs

In [ ]:
%matplotlib inline

ax = gs.plot()

<div class="alert-warning">

Exercício:

1. Usando o notebook anterio coloque esses pontos num mapa estático.

</div>

In [ ]:
west, south, east, north = gs.total_bounds

In [ ]:
%load exe03.py

### Linhas

In [ ]:
rivers = geopandas.read_file(
    f'zip://{datapath.joinpath("ne_50m_rivers_lake_centerlines.zip")}'
)

amazon = rivers[rivers['name'] == 'Amazonas'].geometry.squeeze()

amazon

In [ ]:
type(amazon)

### Polígonos

In [ ]:
countries = geopandas.read_file(
    f'zip://{datapath.joinpath("ne_110m_admin_0_countries.zip")}'
)

brasil = countries[countries['name'] == 'Brazil'].geometry.squeeze()

brasil

In [ ]:
type(brasil)

### Vantagens do GeoPandas: operações em alto nível

In [ ]:
intersec = countries[countries.intersects(brasil)]

intersec

<div class="alert-warning">

Exercício:

1. Leia o help do método `crosses` e crie uma lista dos países que o Rio Amazonas cruza.

</div>

```Python
countries.crosses?
```

### Lemos GeoJson, zip'ed SIG e agora o notório shapefile.

In [ ]:
oceans = geopandas.read_file(datapath.joinpath("oceans.shp"))
oceans

In [ ]:
oceans.crs

In [ ]:
oceans.geom_type

In [ ]:
oceans.plot(
    column="Oceans",
    categorical=True,
    legend=True,
    figsize=(14, 6)
);

<div class="alert-warning">

Exercício:

1. Qual é a maior bacia oceânica?

Dica: cheque o método `area`.

</div>

```
oceans?
```

In [ ]:
countries.plot(
    ax=oceans.plot(cmap='Set2', figsize=(10, 10)),
    facecolor='gray'
);

In [ ]:
import requests
import geojson


wfs_url = "http://data.nanoos.org/geoserver/ows"
params = {
    "service": "WFS",
    "version": "1.0.0",
    "request": "GetFeature",
    "typeName": "oa:goaoninv",
    "outputFormat": "json",
}

r = requests.get(wfs_url, params=params)
wfs_geo = geojson.loads(r.content)

In [ ]:
gdf = geopandas.GeoDataFrame.from_features(wfs_geo)
gdf

In [ ]:
gdf.iloc[-1]

In [ ]:
gdf.plot(
    ax=countries.plot(
        cmap="Set3",
        figsize=(10, 6)
    ),
    marker="o",
    color="red",
    markersize=15
);

<div class="alert-success">

Lemos dados do disco e online com o geopandas e plotamos em um mapas estáticos.

- Dados vetoriais são em geral menores é mas fácil de representar matematicamete/geometricamente que raster.
- Os tipos são pontos, linhas, polígonos e suas combinações.
- 

Recomendo http://darribas.org/gds19/labs/Lab_03.html e http://darribas.org/gds19/notes/Class_03.html para exemplo de visualização legais e úteis!

</div>